In [1]:
import altair as alt
import pandas as pd
import numpy as np
import warnings 

In [2]:
df = pd.read_csv("../data/season_history/nfl_season_history.csv")
df = df.rename({"Pts":"pts_winner", "Pts.1":"pts_loser"}, axis=1)

df_bears = df[df["Winner/tie"].eq("Chicago Bears") | df["Loser/tie"].eq("Chicago Bears")]

df_bears["bears_score"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears.pts_winner, df_bears.pts_loser) 
df_bears["opp_score"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears.pts_loser, df_bears.pts_winner) 
df_bears["opp"] = np.where(df_bears["Winner/tie"].eq("Chicago Bears"), df_bears["Loser/tie"], df_bears["Winner/tie"]) 
df_bears["score_diff"] = df_bears.bears_score - df_bears.opp_score
df_bears["result"] = np.where(df_bears.score_diff <0, "L", "W")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [3]:
df_bears

,Unnamed: 0,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,pts_winner,...,YdsW,TOW,YdsL,TOL,season_year,bears_score,opp_score,opp,score_diff,result
4,4,1,Sun,2020-09-13,1:00PM,Chicago Bears,@,Detroit Lions,boxscore,27.0,...,363.0,0.0,426.0,1.0,2020,27.0,23.0,Detroit Lions,4.0,W
20,20,2,Sun,2020-09-20,1:00PM,Chicago Bears,NaN,New York Giants,boxscore,17.0,...,304.0,2.0,295.0,2.0,2020,17.0,13.0,New York Giants,4.0,W
33,33,3,Sun,2020-09-27,1:00PM,Chicago Bears,@,Atlanta Falcons,boxscore,30.0,...,437.0,2.0,371.0,1.0,2020,30.0,26.0,Atlanta Falcons,4.0,W
50,50,4,Sun,2020-10-04,1:00PM,Indianapolis Colts,@,Chicago Bears,boxscore,19.0,...,289.0,0.0,269.0,1.0,2020,11.0,19.0,Indianapolis Colts,-8.0,L
63,63,5,Thu,2020-10-08,8:20PM,Chicago Bears,NaN,Tampa Bay Buccaneers,boxscore,20.0,...,243.0,1.0,339.0,1.0,2020,20.0,19.0,Tampa Bay Buccaneers,1.0,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12530,122,10,Sun,1973-11-18,2:00PM,Detroit Lions,@,Chicago Bears,boxscore,30.0,...,228.0,2.0,258.0,5.0,1973,7.0,30.0,Detroit Lions,-23.0,L
12544,136,11,Sun,1973-11-25,2:00PM,Minnesota Vikings,NaN,Chicago Bears,boxscore,31.0,...,375.0,3.0,198.0,2.0,1973,13.0,31.0,Minnesota Vikings,-18.0,L
12556,148,12,Sun,1973-12-02,2:00PM,Los Angeles Rams,@,Chicago Bears,boxscore,26.0,...,386.0,2.0,100.0,2.0,1973,0.0,26.0,Los Angeles Rams,-26.0,L
12568,160,13,Sun,1973-12-09,1:00PM,Detroit Lions,NaN,Chicago Bears,boxscore,40.0,...,306.0,1.0,79.0,2.0,1973,7.0,40.0,Detroit Lions,-33.0,L


In [4]:
chart_title = "1990 - 2021 Bears Season Heatmap"
y_cfgs = {"shorthand":"Week:O", "title": "Week"}
x_cfgs = {"shorthand":"season_year:O", "title":"Year"}
# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"

In [5]:
playoff_games = ["WildCard", "Division", "ConfChamp", "SuperBowl"]


In [6]:
alt.Chart(df_bears, width=1500, height=900, title=chart_title).mark_rect()\
.encode(
    y=alt.Y(**y_cfgs,  sort=playoff_games[::-1] + [str(i) for i in range(19)][::-1]),
    x=alt.X(**x_cfgs),
    color=alt.Color(
        "score_diff:Q", 
        scale=alt.Scale(range=['red','white','green'], domainMid=0), 
        legend=alt.Legend(title='Bears Win/Loss Margin'),
    ),
    tooltip=[alt.Tooltip("opp", title="Opponent Name"), 
             "Week", 
             alt.Tooltip("season_year", title="Season Year"),
            alt.Tooltip("bears_score", title="Bears Score"),
            alt.Tooltip("opp_score", title="Opponent Score"),
            alt.Tooltip("result", title="Result"),     
            ]
)\
.configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=25, titleFontSize=20)\
.configure_axisBottom(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=25, titleFontSize=17)\
.configure_title(color=secondary_color, fontSize=25)\
.configure_legend(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10,cornerRadius=10,orient='top-right', labelFontSize=15, titleFontSize=15)


alt.Chart(...)

In [12]:

df_bears.sort_values("score_diff")

,Unnamed: 0,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,pts_winner,...,YdsW,TOW,YdsL,TOL,season_year,bears_score,opp_score,opp,score_diff,result
1580,238,16,Sun,2013-12-22,8:30PM,Philadelphia Eagles,NaN,Chicago Bears,boxscore,54.0,...,514.0,1.0,257.0,2.0,2013,11.0,54.0,Philadelphia Eagles,-43.0,L
6199,13,1,Sun,2003-09-07,4:19PM,San Francisco 49ers,NaN,Chicago Bears,boxscore,49.0,...,391.0,2.0,127.0,5.0,2003,7.0,49.0,San Francisco 49ers,-42.0,L
5417,19,2,Sun,2000-09-10,1:01PM,Tampa Bay Buccaneers,NaN,Chicago Bears,boxscore,41.0,...,327.0,0.0,165.0,4.0,2000,0.0,41.0,Tampa Bay Buccaneers,-41.0,L
1219,145,10,Sun,2014-11-09,8:30PM,Green Bay Packers,NaN,Chicago Bears,boxscore,55.0,...,451.0,1.0,311.0,3.0,2014,14.0,55.0,Green Bay Packers,-41.0,L
3409,223,17,Mon,1991-12-23,9:00PM,San Francisco 49ers,NaN,Chicago Bears,boxscore,52.0,...,486.0,1.0,404.0,3.0,1991,14.0,52.0,San Francisco 49ers,-38.0,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,126,9,Sun,2018-11-04,1:00PM,Chicago Bears,@,Buffalo Bills,boxscore,41.0,...,190.0,1.0,264.0,4.0,2018,41.0,9.0,Buffalo Bills,32.0,W
6741,19,2,Sun,2005-09-18,1:02PM,Chicago Bears,NaN,Detroit Lions,boxscore,38.0,...,336.0,0.0,234.0,5.0,2005,38.0,6.0,Detroit Lions,32.0,W
7055,65,5,Sun,2006-10-08,1:03PM,Chicago Bears,NaN,Buffalo Bills,boxscore,40.0,...,351.0,1.0,145.0,5.0,2006,40.0,7.0,Buffalo Bills,33.0,W
589,51,4,Sun,2018-09-30,1:00PM,Chicago Bears,NaN,Tampa Bay Buccaneers,boxscore,48.0,...,483.0,0.0,311.0,3.0,2018,48.0,10.0,Tampa Bay Buccaneers,38.0,W
